#### Introduction to Statistical Learning, Exercise 3.6

__Please do yourself a favour and only look at the solutions after you honestly tried to solve the exercises.__

# Boston Data Set, Automation

We will further investigate the `Boston` data set, trying to predict the crime rate. We are going to fit a lot of models and compare the results. This requires some automation, ie. writing some loops and such.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from islpy import datasets, utils, lmplots
sns.set()
%matplotlib inline

### A. Regression for each Predictor

For each predictor, fit a *simple linear regression* model with `crim` as the response. Describe your results. In which of the models do you find a statistically significant relationship between the predictor and the response? Produce some plots to back up your assertions.



In [ ]:
boston = datasets.Boston()
boston.head()

In [ ]:
lms = {}
for pred in boston.columns.drop('crim'):
    lms[pred] = smf.ols(f'crim~{pred}', boston).fit()

In [ ]:
ps = [lm.pvalues[1] for lm in lms.values()]
fs = [lm.fvalue for lm in lms.values()]
coeff = [lm.params[1] for lm in lms.values()]
err = [lm.bse[1] for lm in lms.values()]

results = pd.DataFrame({'p_value': ps, 'f_stat': fs, 'coeff': coeff, 'err': err},
                       index=lms.keys())
results.sort_values(by='p_value')

According to the $p$-values, we can reject the null hypothesis $H_0: \beta_1 = 0$ for all but one (`chas`) of the models.

As expected in this scenario, we observe high $F$-statistic values for low $p$-values.

As examples, we plot the fit results of the `lstat` and `nox` model.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4.5))
lmplots.plot_fit(lms['lstat'], 'lstat', ax=ax[0], show_pi=True, lowess=True, legend=True)
lmplots.plot_fit(lms['nox'], 'nox', ax=ax[1], show_pi=True, lowess=True, legend=True)
plt.show()

### B. Multiple Linear Regression

Fit a multiple linear regression model to the `Boston` data set with `crim` as the response and all other variables as predictors. For which predictors can we reject the null Hypothesis $H_0: \beta_j = 0$?

In [ ]:
formula = 'crim~' + '+'.join(boston.columns.drop('crim'))
lm = smf.ols(formula, boston).fit()
lm.pvalues[lm.pvalues.lt(0.05)].sort_values()

We can reject the null hypothesis $H_0: \beta_j = 0$ for `rad`, `dis`, `medv`, `zn` and `black` with 95% confidence. However, the evidence for $\beta_\mathrm{black} \ne 0$ is much weaker than for the others.

### C. Coefficient Comparison

How do your results from __A__ compare to your results from __B__?

Produce a scatter plot of the coefficients from __A__ versus the coefficients from __B__, ignoring the intercept. That is, the plot should have one point for each predictor with the $x$/$y$ coordinates being the coefficients from __B__/__A__.  

In the multiple linear regression a lot of the coefficients have $p$-values > 0.05. Compared the results from __A__ this suggests the presence of strong correlations among the predictors.

In [ ]:
coeff_a = [m.params[1] for m in lms.values()]
coeff_b = lm.params[1:]
ax = sns.scatterplot(coeff_b, coeff_a)
ax.set_xlabel('Multiple Linear Regression')
ax.set_ylabel('Simple Linear Regression')
for name, x, y in zip(boston.columns.drop('crim'), coeff_b, coeff_a):
    ax.annotate(name, (x, y))

The coefficient for `nox` is curious: it changed from strongly positive to strongly negative.

### D. Non-linearity

Is there evidence of non-linear relations to the response for any of the predictors? To answer this question, for each predictor $X$, fit a model of the form

$$ Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 +\epsilon$$

In [ ]:
lms = {}
for pred in boston.columns.drop('crim'):
    lm = smf.ols(f'crim~{pred}+I({pred}**2)+I({pred}**3)', boston).fit()
    lms[pred] = lm

In [ ]:
results = pd.DataFrame(np.zeros((boston.shape[1] - 1, 6)), columns=[
                 'c1', 'c2', 'c3', 'p1', 'p2', 'p3'])
results.set_index(boston.columns.drop('crim'), inplace=True)
buffer = np.zeros(6)
for pred in lms:
    lm = lms[pred]
    buffer[:3] = lm.params[1:].values
    buffer[3:] = lm.pvalues[1:].values
    results.loc[pred] = buffer
    
results[(results.p2 < 0.05) | (results.p3 < 0.05)]

The table above shows the coefficients and $p$-values for the predictors that show evidence for non-linear relationships to the response. We plot the fit results for two of them. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))
ax = lmplots.plot_fit(lms['nox'], 'nox', ax=ax1)
ax = lmplots.plot_fit(lms['medv'], 'medv', ax=ax2)